Training Model

In [ ]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# change to working directory on the drive
%cd '/content/gdrive/My Drive/'

In [ ]:
#Read the data
df=pd.read_csv('news.csv')
#Get shape and head
df.shape
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
#DataFlair - Get the labels
labels=df.label
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [ ]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=train_test_split(df['text'], labels, test_size=0.2, random_state=7)

In [ ]:
#DataFlair - Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
#DataFlair - Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [ ]:
#DataFlair - Initialize a PassiveAggressiveClassifier
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)
#DataFlair - Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 92.58%


In [ ]:
#DataFlair - Build confusion matrix
confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])

array([[589,  49],
       [ 42, 587]])

Twitter API Consume

In [ ]:
!pip install tweepy

In [ ]:
import tweepy
from textblob import TextBlob
import pandas as pd
import re

In [ ]:
API_KEY = "XC8RN7qtQH9apkyqdOv1WkPeR"
API_SECRET = "08y6a6mMIfKe3mIXXLEeYs609LbNAF5pW9U1b2UNiUjXqwRIPq"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAN7MVwEAAAAAXQtFdnPo3WMUEBLv2XsxEW7a4R8%3DZLfz63tW8P4P2N9tKMkhxAClEPvFQw8wKqFANzNRlq5uQ6nI3U"
ACCESS_TOKEN = "596090159-3KzqC9ndCEtGTE4aQbleluOSUDid0G8uQ3WXz3VB"
ACCESS_TOKEN_SECRET = "c9DANcKaOkH2Yvs4yOfQRb6Drb0yRJdPpgiDwTctoBEYZ"

In [ ]:
auth = tweepy.OAuthHandler(API_KEY,API_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [ ]:
# result = api.user_timeline(id="covid19",lang="en",count=1000)
result = api.search(q="covid",lang="en",count=100)

In [ ]:
result

[Status(_api=<tweepy.api.API object at 0x7fdea03d18d0>, _json={'created_at': 'Wed Dec 01 16:07:37 +0000 2021', 'id': 1466076528995979271, 'id_str': '1466076528995979271', 'text': 'RT @PoliticsForAlI: 🚨🚨🚨 | BREAKING: The Omicron variant appears to be a ‘super mild‘ mutation with Covid death rate in South Africa not jum…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'PoliticsForAlI', 'name': 'Politics For All', 'id': 1166419406685188101, 'id_str': '1166419406685188101', 'indices': [3, 18]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 162490837, 'id_str': '162490837', 'name': 'Kieran Morgan', 'screen_name': 'kcceleb', 'location': '

In [ ]:
tweet_data = []
for tweet in result:
  tweet_property={}
  tweet_property["tanggal_tweet"] = tweet.created_at
  tweet_property["pengguna"] = tweet.user.screen_name
  tweet_property["isi_tweet"] = tweet.text
  tweet_property["tweet_bersih"] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet.text).split())
  tweet_data.append(tweet_property)

tweet_data

[{'isi_tweet': 'RT @PoliticsForAlI: 🚨🚨🚨 | BREAKING: The Omicron variant appears to be a ‘super mild‘ mutation with Covid death rate in South Africa not jum…',
  'pengguna': 'kcceleb',
  'tanggal_tweet': datetime.datetime(2021, 12, 1, 16, 7, 37),
  'tweet_bersih': 'RT BREAKING The Omicron variant appears to be a super mild mutation with Covid death rate in South Africa not jum'},
 {'isi_tweet': 'RT @MeidasTouch: Trump was single-handedly the biggest Covid superspreader in the world. And he infected people on purpose. What a filthy,…',
  'pengguna': 'soapdemon34',
  'tanggal_tweet': datetime.datetime(2021, 12, 1, 16, 7, 37),
  'tweet_bersih': 'RT Trump was single handedly the biggest Covid superspreader in the world And he infected people on purpose What a filthy'},
 {'isi_tweet': 'RT @MeidasTouch: Trump was single-handedly the biggest Covid superspreader in the world. And he infected people on purpose. What a filthy,…',
  'pengguna': 'WesfromChgo',
  'tanggal_tweet': datetime.datetime(2

In [ ]:
def findLabel(text):
  vec_new_test = tfidf_vectorizer.transform([text])
  y_pred = pac.predict(vec_new_test)
  return y_pred

In [ ]:
fake = 0
real = 0
for i in tweet_data:
  result_scanning = findLabel(i["tweet_bersih"])
  print("Data tweet =",i["tweet_bersih"])
  print("Hasil analisa =",result_scanning[0],"\n")
  if result_scanning[0]=='FAKE':
    fake+=1
  else:
    real+=1

print ("Total fake news =",fake)
print ("Total real news =",real)

Data tweet = RT BREAKING The Omicron variant appears to be a super mild mutation with Covid death rate in South Africa not jum
Hasil analisa = FAKE 

Data tweet = RT Trump was single handedly the biggest Covid superspreader in the world And he infected people on purpose What a filthy
Hasil analisa = FAKE 

Data tweet = RT Trump was single handedly the biggest Covid superspreader in the world And he infected people on purpose What a filthy
Hasil analisa = FAKE 

Data tweet = RT LeBron James fully vaccinated has contracted COVID Kyrie Irving not vaccinated has not contracted COVID
Hasil analisa = FAKE 

Data tweet = RT I can t blame the public for being confused and angry about COVID information they re hearing The sheer amount of data com
Hasil analisa = FAKE 

Data tweet = RT Remember the sense of shock when Trump admitted he was Covid positive and then was rushed to Walter Reed 18 hours later
Hasil analisa = FAKE 

Data tweet = RT EXPLOSIVE We now learn Trump had tested positive for C